# Feature Engineering

In [1]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, ArrayType, StringType
from pyspark.sql.functions import col, when, size, expr, udf

import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

spark = SparkSession.builder.appName("Forum Question Analyzer") \
    .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.17.0") \
    .getOrCreate()
posts = spark.read \
    .format("com.databricks.spark.xml") \
    .option("rootTag", "posts") \
    .option("rowTag", "row") \
    .load("tex.stackexchange.com/Posts.xml") \
    .alias('posts')
users = spark.read \
    .format("com.databricks.spark.xml") \
    .option("rootTag", "users") \
    .option("rowTag", "row") \
    .load("tex.stackexchange.com/Users.xml") \
    .alias('users')
tags = spark.read \
    .format("com.databricks.spark.xml") \
    .option("rootTag", "tags") \
    .option("rowTag", "row") \
    .load("tex.stackexchange.com/Tags.xml") \
    .alias('tags')

In [2]:
data = posts.filter(posts._PostTypeId == 1)
data = data.withColumn("accepted", when(col("_AcceptedAnswerId").isNull(), 0).otherwise(1))

In [3]:
posts = posts.filter(posts._PostTypeId==1)
questions = posts.withColumn("_Tags", expr("split(substring(_Tags, 2, length(_Tags) - 2), '><')"))\
            .withColumn("_Body", size(expr("split(_Body, ' ')")))\
            .withColumn("_Title", size(expr("split(_Title, ' ')")))

questions = questions.join(users, questions._OwnerUserId == users._Id).select(
    questions._Id.alias("QuestionId"),
    questions._Body.alias("BodyLength"),
    questions._Title.alias("TitleLength"),
    questions._Tags.alias("TagsCounts"),
    size(questions._Tags).alias("NumberOfTags"),
    users._Id.alias("OwnerId"),
    users._DownVotes.alias("OwnerDownVotes"),
    users._UpVotes.alias("OwnerUpVotes"),
    users._Reputation.alias("OwnerReputation"),
    users._Views.alias("OwnerViews"),
    (questions._CreationDate - users._CreationDate).cast("integer").alias("UserExperience"),
    when(col("_AcceptedAnswerId").isNull(), 0).otherwise(1).alias("Accepted")
)

#Fast way to change tag names to tag counts from Tags table
#####
tag_counts = tags.select("_TagName", "_Count").rdd.collectAsMap()

def replace_tags_with_counts(tags):
    return [tag_counts.get(tag, 0) for tag in tags]

replace_tags_with_counts_udf = udf(replace_tags_with_counts, ArrayType(IntegerType()))
#####

questions = questions.withColumn("TagsCounts", replace_tags_with_counts_udf(questions.TagsCounts))

## Feature importances

In [4]:
print(questions.columns)

['QuestionId', 'BodyLength', 'TitleLength', 'TagsCounts', 'NumberOfTags', 'OwnerId', 'OwnerDownVotes', 'OwnerUpVotes', 'OwnerReputation', 'OwnerViews', 'UserExperience', 'Accepted']


In [5]:
features = ['BodyLength', 'TitleLength', 'NumberOfTags', 'OwnerUpVotes', 'OwnerDownVotes']
assembler = VectorAssembler(inputCols=features, outputCol="features")

In [8]:
rf = RandomForestClassifier(labelCol="Accepted", featuresCol="features", numTrees=10)
rf_pipeline = Pipeline(stages=[assembler, rf])
rf_model = rf_pipeline.fit(questions)
result = rf_model.stages[-1].featureImportances.toArray()

In [9]:
print("Feature Importances:")
for feature, importance in zip(features, sorted(result, reverse=True)):
    print(f"{feature}: {importance}")

Feature Importances:
BodyLength: 0.8602048456049463
TitleLength: 0.12749007296012757
NumberOfTags: 0.009404382545718425
OwnerUpVotes: 0.0018867962833272335
OwnerDownVotes: 0.0010139026058803451
